In [ ]:
import pandas as pd
from pandas.api.types import is_string_dtype


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from pandas.api.types import is_string_dtype, is_integer_dtype, is_float_dtype, is_bool_dtype

tables_order = [
    "Image", "Continent", "Location", "Festival", "Stage", "Event", "Equipment",
    "Role", "Experience", "Staff", "Performer", "Performance_type", "Performance",
    "Artist", "Visitor", "Artist_has_Performer", "Ticket_type", "Payment_method",
    "Ticket", "ResaleQueue", "Likert", "Rating", "Event_has_Staff", "Genre",
    "Subgenre", "Performer_has_Subgenre", "Stage_has_Equipment", "Visitor_wants_Ticket"
]

queries = [
    "USE mydb;",
    "SET SESSION sql_mode='NO_AUTO_VALUE_ON_ZERO';",
    "SET FOREIGN_KEY_CHECKS=0;"
]

for table in tables_order:
    try:
        df = pd.read_csv(f"drive/MyDrive/festival_data_new/{table}.csv")

        # Convert NaN/NA values to NULL
        df = df.where(pd.notnull(df), None)

        columns = ", ".join([f"`{col}`" for col in df.columns])

        # Prepare the INSERT statement with proper value formatting
        sql = f"INSERT INTO `{table.lower()}` ({columns}) VALUES ("

        for row in df.itertuples(index=False, name=None):
            values = []
            for val in row:
                if val is None:
                    values.append("NULL")
                elif isinstance(val, str):
                    # Escape single quotes and wrap in quotes
                    val_escaped = val.replace("'", "''")
                    values.append(f"'{val_escaped}'")
                elif isinstance(val, (int, float)) and not pd.isna(val):
                    values.append(str(val))
                elif isinstance(val, bool):
                    values.append(str(int(val)))
                else:
                    values.append("NULL")

            query = sql + ", ".join(values) + ");"
            queries.append(query)

    except FileNotFoundError:
        print(f"Warning: File not found for table {table} - skipping")
    except Exception as e:
        print(f"Error processing table {table}: {str(e)}")

queries.append("SET FOREIGN_KEY_CHECKS=1;")

with open("dml_script_fixed.sql", "w", encoding="utf-8") as f:
    for query in queries:
        f.write(query + "\n")

print("Fixed DML script generated successfully as dml_script_fixed.sql")

Fixed DML script generated successfully as dml_script_fixed.sql
